In [ ]:
with open('/content/drive/MyDrive/txt', 'r', encoding='utf-8') as f:
    train_text = f.read()
len(train_text)

In [ ]:
tokens=train_text.encode("utf-8")
tokens=list(map(int,tokens))
len(tokens)

In [ ]:
def get_stats(ids):
  counts={}
  for pair in zip(ids, ids[1:]):
    counts[pair]=counts.get(pair,0)+1
  return counts

stats=get_stats(tokens)

def merge(ids,pair,idx):
  new_ids=[]
  i=0
  while i<len(ids):
    if i<len(ids)-1 and ids[i]==pair[0] and ids[i+1]==pair[1]:
      new_ids.append(idx)
      i+=2
    else:
      new_ids.append(ids[i])
      i+=1
  return new_ids

vocab_size=1500
merges_num=vocab_size-256
ids=list(tokens)
merges={}
for i in range (merges_num):
  stats=get_stats(ids)
  pair=max(stats,key=stats.get)
  idx=256+i
  print(f"merging {pair} into new token {idx}")
  ids=merge(ids,pair,idx)
  merges[pair]=idx
print(ids)
print(len(ids))
print(f"compression ration: {len(tokens)/len(ids)}")
print(merges)

In [ ]:
#decoding
vocab={idx: bytes([idx]) for idx in range(256)}
for (p0,p1), idx in merges.items():
  vocab[idx]=vocab[p0]+vocab[p1]
# print(vocab)
def decode(ids):
  tokens=b"".join(vocab[idx] for idx in ids)
  text=tokens.decode("utf-8", errors="replace")
  return text


#encoding
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

# print(decode(encode("hello world")))

In [ ]:
with open('/content/drive/MyDrive/alllines.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
import regex as re
gpt2pat = re.compile(r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,3}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""")
a=re.findall(gpt2pat, text)
g=[]
for i in a:
  s=encode(i)
  g=g+s
print(len(g))

In [ ]:
list_str = ', '.join(map(str, g))  # Convert each element to a string and join them with a comma

# Specify the file path
file_path = 'list_data.txt'

# Write the string representation of the list to a file
with open(file_path, 'w') as file:
    file.write(list_str)

print("List saved to", file_path)

In [ ]:
 data = torch.tensor(g, dtype=torch.long)
 type(data)

In [ ]:
data = torch.tensor(g, dtype=torch.long)
n = int(0.85*len(data))
train_data = data[:n]
val_data = data[n:]
block_size = 8
train_data[:block_size+1]
import torch
block_size = 8

def generate_context_target_pairs(x, block_size):
    """Generate and print context-target pairs from a given sequence."""
    for t in range(block_size):
        context = x[:t+1]
        target = x[t+1] if t + 1 < len(x) else None
        print(f"when input is {context} the target: {target}")

x = train_data[:block_size]
generate_context_target_pairs(x, block_size)

In [ ]:
block_size = 8
train_data[:block_size+1]
import torch
block_size = 8

def generate_context_target_pairs(x, block_size):
    """Generate and print context-target pairs from a given sequence."""
    for t in range(block_size):
        context = x[:t+1]
        target = x[t+1] if t + 1 < len(x) else None
        print(f"when input is {context} the target: {target}")

x = train_data[:block_size]
generate_context_target_pairs(x, block_size)

In [ ]:
# We are not feeding all the data directly into the model so but making a batch and feeding it batwise for memory management

import torch

torch.manual_seed(1337)
batch_size = 4



def get_batch(data, batch_size, block_size):
    """Generate a batch of data for inputs x and targets y."""
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

def print_batch_details(xb, yb):
    """Print details of the input and target batches."""
    print('inputs:')
    print(xb.shape)
    print(xb)
    print('targets:')
    print(yb.shape)
    print(yb)
    print('----')

    for b in range(batch_size):
        for t in range(block_size):
            context = xb[b, :t+1].tolist()
            target = yb[b, t].item()
            print(f"when input is {context} the target: {target}")




xb, yb = get_batch(train_data, batch_size, block_size)
print_batch_details(xb, yb)


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 #
block_size = 256 #
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        perplexities = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, perplexity = model(X, Y)
            losses[k] = loss.item()
            perplexities[k] = perplexity.item() if perplexity is not None else float('nan')
        out[f'{split}_loss'] = losses.mean()
        out[f'{split}_perplexity'] = perplexities.mean()
    model.train()
    return out

def save_checkpoint(model, optimizer, iter, filepath):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'iteration': iter
    }
    torch.save(checkpoint, filepath)
    print(f"Model checkpoint saved at iteration {iter}.")


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key=nn.Linear(n_embd, head_size, bias=False)
        self.query=nn.Linear(n_embd, head_size, bias=False)
        self.value=nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C=x.shape
        k=self.key(x)   # (B,T,hs)
        q=self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei=q@k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei=wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei=F.softmax(wei, dim=-1)
        wei=self.dropout(wei)
        v=self.value(x)
        out=wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(head_size * num_heads,n_embd)
        self.dropout=nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size=n_embd // n_head
        self.sa=MultiHeadAttention(n_head, head_size)
        self.ffwd=FeedFoward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table=nn.Embedding(block_size, n_embd)
        self.blocks=nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) # final layer norm
        self.lm_head=nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T =idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb=self.token_embedding_table(idx)
        pos_emb=self.position_embedding_table(torch.arange(T, device=device))
        x=tok_emb+pos_emb
        x=self.blocks(x)
        x=self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss=None
            perplexity=None
        else:
            B,T,C =logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
            probabilities=F.softmax(logits, dim=-1)
            log_probabilities=torch.log(probabilities)
            log_prob=torch.gather(log_probabilities,1,targets.unsqueeze(1)).squeeze(1)
            N=targets.size(0)
            perplexity=torch.exp(-torch.sum(log_prob)/N)

        return logits, loss,perplexity

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond=idx[:, -block_size:]
            # get the predictions
            logits,loss,perplexity = self(idx_cond)
            # focus only on the last time step
            logits=logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs=F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next=torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx =torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


optimizer =torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):


    if iter % eval_interval ==0 or iter==max_iters - 1:
        losses=estimate_loss()
        print(f"step {iter}: train loss {losses['train_loss']:.4f}, val loss {losses['val_loss']:.4f}, PERPLEX_TRAIN {losses['train_perplexity']:.4f} "+
              f"PERPLEX_VAL {losses['val_perplexity']:.4f}"
        )

        save_checkpoint(model, optimizer, iter, f"checkpoint_iteration_{iter}.pt")


    # sample a batch of data
    xb, yb =get_batch('train')

    # evaluate the loss
    logits,loss,perplexity = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()



In [ ]:
with open('/content/drive/MyDrive/list_data_for_vocab_size_1500.txt', 'r', encoding='utf-8') as f:
    text = f.read()
len(text)

In [ ]:
numbers_as_strings = text.split(',')
numbers = [int(num_str.strip()) for num_str in numbers_as_strings]

In [ ]:
import torch
data = torch.tensor(numbers, dtype=torch.long)
n = int(0.85*len(data))
train_data = data[:n]
val_data = data[n:]
block_size = 8
train_data[:block_size+1]

def generate_context_target_pairs(x, block_size):
    """Generate and print context-target pairs from a given sequence."""
    for t in range(block_size):
        context = x[:t+1]
        target = x[t+1] if t + 1 < len(x) else None
        print(f"when input is {context} the target: {target}")

x = train_data[:block_size]
generate_context_target_pairs(x, block_size)
# We are not feeding all the data directly into the model so but making a batch and feeding it batwise for memory management

import torch

torch.manual_seed(1337)
batch_size = 4



def get_batch(data, batch_size, block_size):
    """Generate a batch of data for inputs x and targets y."""
    ix = torch.randint(0, len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

def print_batch_details(xb, yb):
    """Print details of the input and target batches."""
    print('inputs:')
    print(xb.shape)
    print(xb)
    print('targets:')
    print(yb.shape)
    print(yb)
    print('----')

    for b in range(batch_size):
        for t in range(block_size):
            context = xb[b, :t+1].tolist()
            target = yb[b, t].item()
            print(f"when input is {context} the target: {target}")


xb, yb = get_batch(train_data, batch_size, block_size)
print_batch_details(xb, yb)


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 64 #
block_size = 256 #
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
vocab_size=1500


# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        perplexities = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss, perplexity = model(X, Y)
            losses[k] = loss.item()
            perplexities[k] = perplexity.item() if perplexity is not None else float('nan')
        out[f'{split}_loss'] = losses.mean()
        out[f'{split}_perplexity'] = perplexities.mean()
    model.train()
    return out

def save_checkpoint(model, optimizer, iter, filepath):
    checkpoint = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'iteration': iter
    }
    torch.save(checkpoint, filepath)
    print(f"Model checkpoint saved at iteration {iter}.")


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key=nn.Linear(n_embd, head_size, bias=False)
        self.query=nn.Linear(n_embd, head_size, bias=False)
        self.value=nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C=x.shape
        k=self.key(x)   # (B,T,hs)
        q=self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei=q@k.transpose(-2,-1) * k.shape[-1]**-0.5
        wei=wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei=F.softmax(wei, dim=-1)
        wei=self.dropout(wei)
        v=self.value(x)
        out=wei @ v
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads=nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj=nn.Linear(head_size * num_heads,n_embd)
        self.dropout=nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net=nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size=n_embd // n_head
        self.sa=MultiHeadAttention(n_head, head_size)
        self.ffwd=FeedFoward(n_embd)
        self.ln1=nn.LayerNorm(n_embd)
        self.ln2=nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table=nn.Embedding(block_size, n_embd)
        self.blocks=nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f=nn.LayerNorm(n_embd) # final layer norm
        self.lm_head=nn.Linear(n_embd, vocab_size)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T =idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb=self.token_embedding_table(idx)
        pos_emb=self.position_embedding_table(torch.arange(T, device=device))
        x=tok_emb+pos_emb
        x=self.blocks(x)
        x=self.ln_f(x)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss=None
            perplexity=None
        else:
            B,T,C =logits.shape
            logits=logits.view(B*T, C)
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits, targets)
            probabilities=F.softmax(logits, dim=-1)
            log_probabilities=torch.log(probabilities)
            log_prob=torch.gather(log_probabilities,1,targets.unsqueeze(1)).squeeze(1)
            N=targets.size(0)
            perplexity=torch.exp(-torch.sum(log_prob)/N)

        return logits, loss,perplexity

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond=idx[:, -block_size:]
            # get the predictions
            logits,loss,perplexity = self(idx_cond)
            # focus only on the last time step
            logits=logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs=F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next=torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx =torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx


model=GPTLanguageModel()
model=model.to(device)
checkpoint = torch.load('/content/drive/MyDrive/checkpoint_iteration_14000.pt')#, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer =torch.optim.AdamW(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
context = torch.tensor([[70,339,277,32,67,284,1134,269,58,10,66,352,263,101]],dtype=torch.long,device=device)
summary= decode(model.generate(context,max_new_tokens=2000)[0].tolist())


print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')





model=GPTLanguageModel()
model=model.to(device)
checkpoint = torch.load('/content/drive/MyDrive/checkpoint_iteration_4999.pt')#, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer =torch.optim.AdamW(model.parameters(), lr=learning_rate)
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
